<a href="https://colab.research.google.com/github/deepu8900/Resume_JD_Matcher/blob/main/resume_matcher_project.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [40]:
!pip install -q sentence-transformers faiss-cpu streamlit fastapi uvicorn nest-asyncio pyngrok



In [41]:
from pyngrok import ngrok
ngrok.kill()


In [42]:
%%writefile matcher.py
from sentence_transformers import SentenceTransformer
import numpy as np
import re

model = SentenceTransformer('all-MiniLM-L6-v2')

def preprocess_text(text):
    sentences = re.split(r'[.\n]', text)
    return [s.strip() for s in sentences if s.strip()]

def compute_embeddings(sentences):
    return model.encode(sentences)

def cosine_similarity(a, b):
    return np.dot(a, b) / (np.linalg.norm(a) * np.linalg.norm(b))

def match_resume_to_jd(resume_text, jd_text, threshold=0.75):
    resume_chunks = preprocess_text(resume_text)
    jd_chunks = preprocess_text(jd_text)
    resume_embeddings = compute_embeddings(resume_chunks)
    jd_embeddings = compute_embeddings(jd_chunks)
    matched = []
    missing = []
    for i, res_emb in enumerate(resume_embeddings):
        similarities = [cosine_similarity(res_emb, jd_emb) for jd_emb in jd_embeddings]
        max_sim = max(similarities)
        if max_sim >= threshold:
            matched.append((resume_chunks[i], max_sim))
        else:
            missing.append(resume_chunks[i])
    match_percent = round((len(matched) / len(resume_chunks)) * 100, 2)
    return matched, missing, match_percent


Overwriting matcher.py


In [43]:
%%writefile app.py
import streamlit as st
from matcher import match_resume_to_jd

st.set_page_config(page_title="LLM Resume Matcher", layout="wide")
st.title("📄 Resume vs JD Matcher")

col1, col2 = st.columns(2)

with col1:
    resume_text = st.text_area("Paste Resume Text", height=300)

with col2:
    jd_text = st.text_area("Paste Job Description Text", height=300)

if st.button("🔍 Match Resume to JD"):
    if resume_text and jd_text:
        matched, missing, percentage = match_resume_to_jd(resume_text, jd_text)
        st.success(f"✅ Match Percentage: {percentage}%")
        st.subheader("✔️ Matched Resume Sentences")
        for sentence, score in matched:
            st.markdown(f"- {sentence} _(score: {round(score, 2)})_")
        st.subheader("❌ Missing Sentences")
        for sentence in missing:
            st.markdown(f"- {sentence}")
    else:
        st.warning("Please provide both resume and job description.")


Overwriting app.py


In [44]:
import nest_asyncio
import threading
import time
from pyngrok import ngrok

ngrok.set_auth_token("30s2LdaMd7AWlMnTnBE6n7VNzKZ_3vnYzx8Cox6X6GhzCd942")

def run_streamlit():
    !streamlit run app.py &>/content/log.txt

nest_asyncio.apply()
threading.Thread(target=run_streamlit).start()

time.sleep(5)
public_url = ngrok.connect(8501)
print(f"🚀 Your Streamlit app is live at: {public_url}")


🚀 Your Streamlit app is live at: NgrokTunnel: "https://cb8e20d043cf.ngrok-free.app" -> "http://localhost:8501"
